In [ ]:
from google.colab import files
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install -q kaggle
!kaggle competitions download -c dogs-vs-cats


In [ ]:
!mkdir -p data/train


In [ ]:
!unzip -q dogs-vs-cats.zip -d data
!unzip -q data/train.zip -d data/train


In [ ]:
import os, shutil

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
cat_dir = os.path.join(base_dir, 'cats')
dog_dir = os.path.join(base_dir, 'dogs')

os.makedirs(cat_dir, exist_ok=True)
os.makedirs(dog_dir, exist_ok=True)

for fname in os.listdir(train_dir):
    if fname.startswith('cat'):
        shutil.move(os.path.join(train_dir, fname), os.path.join(cat_dir, fname))
    elif fname.startswith('dog'):
        shutil.move(os.path.join(train_dir, fname), os.path.join(dog_dir, fname))



In [ ]:
from sklearn.model_selection import train_test_split

def split_data(source_dir, train_dir, val_dir, split_size=0.8):
    files = os.listdir(source_dir)
    train_files, val_files = train_test_split(files, train_size=split_size, random_state=42)

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    for file in train_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(train_dir, file))
    for file in val_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(val_dir, file))

split_data(cat_dir, 'data/train/cats', 'data/val/cats')
split_data(dog_dir, 'data/train/dogs', 'data/val/dogs')



In [ ]:
import os, shutil

wrong_dir = 'data/train/train'
correct_dir = 'data/train'

for fname in os.listdir(wrong_dir):
    shutil.move(os.path.join(wrong_dir, fname), os.path.join(correct_dir, fname))

os.rmdir(wrong_dir)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    'data/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    'data/val',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)


In [ ]:
model.save("cats_vs_dogs_model.h5")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!cp cats_vs_dogs_model.h5 /content/drive/MyDrive/


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sample_batch = next(iter(val_data))
images, labels = sample_batch

predictions = model.predict(images)
predicted_classes = (predictions > 0.5).astype("int32")

plt.figure(figsize=(12, 8))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(images[i])
    plt.title(f"Gerçek: {'Dog' if labels[i] else 'Cat'}\nTahmin: {'Dog' if predicted_classes[i] else 'Cat'}")
    plt.axis('off')

plt.tight_layout()
plt.show()
